<a href="https://colab.research.google.com/github/ishankarve/meta_analysis/blob/main/met_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import libraries
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve
%matplotlib inline

In [4]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
# import data
train_df = pd.read_csv("https://raw.githubusercontent.com/ishankarve/meta_analysis/main/metadata.csv")

In [10]:
# prevent result truncation
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 300

In [11]:
train_df.shape

(320, 16)

In [12]:
train_df.head()

,DATE,TIME,F/L (ft),LCL,CCL,LFC,TTI,KI,LI,SI,BI,HI,DCI,SWEAT,CAPE,TS
0,03/06/2019,1630,"16,145",950,933,885,43.0,24.7,-7.3,1.1,95.8,25.3,44.6,-7.9,-,TS
1,04/06/2019,430,"16,790",972,943,895,38.4,27.9,-6.9,4.5,95.9,20.5,36.3,26.9,-,NaN
2,11/08/2019,1630,"16,585",938,-,-,44.7,28.4,-5.5,-1.0,95.1,20.7,45.1,76.2,-,NaN
3,12/08/2019,530,"16,134",996,970,970,41.8,30.3,-4.0,0.4,93.5,10.4,37.6,157.0,1358.9,NaN
4,14/08/2019,530,"16,401",958,950,947,42.3,29.0,-4.3,0.3,98.5,14.2,41.0,208.8,-,NaN


In [13]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
TIME,320.0,1011.906250,641.207563,330.0,430.00,530.0,1730.000,2330.0
TTI,320.0,39.417500,5.269693,18.8,37.60,40.0,42.325,51.9
KI,320.0,21.083750,14.846174,-30.5,11.00,26.8,32.025,66.4
LI,320.0,-1.414375,3.635610,-10.5,-3.60,-2.0,0.700,14.3
SI,320.0,2.796563,3.385015,-16.7,0.80,2.4,4.400,14.9
BI,318.0,94.564780,6.778698,9.1,93.70,95.3,96.575,103.1
HI,319.0,42.175235,24.546631,0.1,19.10,43.2,61.000,118.0
SWEAT,320.0,-18.380312,165.573275,-714.0,-111.65,1.5,94.875,370.6


In [14]:
train_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      320 non-null    object 
 1   TIME      320 non-null    int64  
 2   F/L (ft)  320 non-null    object 
 3   LCL       320 non-null    object 
 4   CCL       320 non-null    object 
 5   LFC       317 non-null    object 
 6   TTI       320 non-null    float64
 7   KI        320 non-null    float64
 8   LI        320 non-null    float64
 9   SI        320 non-null    float64
 10  BI        318 non-null    float64
 11  HI        319 non-null    float64
 12  DCI       320 non-null    object 
 13  SWEAT     320 non-null    float64
 14  CAPE      319 non-null    object 
 15  TS        45 non-null     object 
dtypes: float64(7), int64(1), object(8)
memory usage: 40.1+ KB
